In [7]:
import os
import pandas as pd

root_path = "../rawdata/internal"
output_path = "../data/prpd/internal"

for subdir in os.listdir(root_path):
    if subdir.startswith('.'):
        continue
    subdir_path = os.path.join(root_path, subdir)
    # Split directory name to determine category
    category_parts = subdir.split("_")
    trigger = category_parts[3]
    gain = category_parts[5].split("dB")[0]
    type_list = "_".join(category_parts[8:])
    category = f"Trigger_{trigger}_Gain_{gain}dB_type_list_{type_list}"

    # Read prpd.txt file
    prpd_file = os.path.join(subdir_path, "prpd.txt")
    if not os.path.exists(prpd_file):  # Check if the file exists
        print(f"Skipping directory {subdir} because prpd.txt does not exist")
        continue

    with open(prpd_file, "r") as file:
        lines = file.readlines()

    # Remove first element from each line
    modified_lines = [line.split()[1:] for line in lines]

    # Concatenate modified data
    concatenated_data = "\n".join([" ".join(line) for line in modified_lines])

    # Save as CSV file
    output_file = os.path.join(output_path, f"{category}.csv")
    with open(output_file, "w") as file:
        file.write(concatenated_data)